In [ ]:
import librosa
import numpy as np
import soundfile as sf
from scipy.interpolate import interp1d
from IPython.display import Audio, display

### First approch (mms)- remove utterance

In [ ]:


# Fonction pour calculer la variance du pitch dans chaque segment en utilisant pYIN
def calculate_pitch_variance(audio, sr, frame_length=2048, hop_length=512):
    # Estimer le pitch avec pYIN
    pitches, voiced_flags, voiced_probs = librosa.pyin(audio, 
                                                       fmin=librosa.note_to_hz('C2'), 
                                                       fmax=librosa.note_to_hz('C7'),
                                                       sr=sr, hop_length=hop_length)
    
    # Récupérer les pitches valides (où la voix est détectée)
    pitches_valid = pitches[~np.isnan(pitches)]
    
    # Calculer la variance des pitches valides
    if len(pitches_valid) > 0:
        return np.var(pitches_valid)
    else:
        return 0

# Fonction pour segmenter l'audio en fonction de la durée et calculer la variance du pitch
def remove_high_variance_segments(audio, sr, threshold=0.1, segment_duration=5.0):
    segment_samples = int(segment_duration * sr)
    num_segments = len(audio) // segment_samples
    
    # Créer une liste pour garder les segments valides
    valid_segments = []
    
    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = (i + 1) * segment_samples
        segment = audio[start_sample:end_sample]
        
        # Calculer la variance du pitch pour le segment
        variance = calculate_pitch_variance(segment, sr)
        
        # Si la variance du pitch est inférieure au seuil, garder le segment
        if variance < threshold:
            valid_segments.append(segment)
    
    # Combiner les segments valides
    return np.concatenate(valid_segments)

# Charger un fichier audio
audio_file = "your_audio_file.wav"  # Remplacez par le chemin de votre fichier audio
audio, sr = librosa.load(audio_file, sr=None)

# Définir le seuil de variance et la durée des segments (par exemple, 5 secondes)
variance_threshold = 100.0  # Ajustez le seuil selon vos besoins
segment_duration = 5.0  # Durée de chaque segment à analyser

# Supprimer les segments avec une variance de pitch trop élevée
clean_audio = remove_high_variance_segments(audio, sr, threshold=variance_threshold, segment_duration=segment_duration)

# Sauvegarder l'audio filtré
sf.write("cleaned_audio_py.wav", clean_audio, sr)


#### Approch-2: supprimer uniquement les hautes variances frequence dans les sous trames(5s)

In [ ]:


# Fonction pour calculer le pitch avec pYIN et identifier les frames valides
def detect_high_variance_segments(audio, sr, frame_length=2048, hop_length=512, threshold=100.0):
    # Estimer le pitch avec pYIN
    pitches, voiced_flags, voiced_probs = librosa.pyin(audio, 
                                                       fmin=librosa.note_to_hz('C2'), 
                                                       fmax=librosa.note_to_hz('C7'),
                                                       sr=sr, hop_length=hop_length)
    
    # Récupérer les frames valides (où la voix est présente)
    valid_pitches = pitches[~np.isnan(pitches)]
    
    # Si pas assez de données valides, garder l'audio inchangé
    if len(valid_pitches) == 0:
        return audio
    
    # Calculer la variance des pitches
    pitch_variance = np.var(valid_pitches)
    
    # Supprimer les frames si la variance dépasse le seuil
    if pitch_variance > threshold:
        # Remplacer la portion audio par du silence si la variance est trop haute
        audio[:] = 0  # Mettre la portion à zéro (silencieux)
    
    return audio

# Fonction pour traiter tout l'audio en segments et supprimer les fragments à haute variance
def process_audio(audio, sr, threshold=100.0, segment_duration=5.0):
    segment_samples = int(segment_duration * sr)
    num_segments = len(audio) // segment_samples
    
    # Créer une liste pour garder les segments traités
    processed_audio = np.array([])

    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = (i + 1) * segment_samples
        segment = audio[start_sample:end_sample]
        
        # Traiter chaque segment pour supprimer les portions avec haute variance de pitch
        processed_segment = detect_high_variance_segments(segment, sr, threshold=threshold)
        
        # Ajouter le segment traité à l'audio final
        processed_audio = np.concatenate((processed_audio, processed_segment))
    
    return audio, processed_audio

# Charger un fichier audio
audio_file = "your_audio_file.wav"  # Remplacez par le chemin de votre fichier audio
audio, sr = librosa.load(audio_file, sr=None)

# Définir le seuil de variance et la durée des segments
variance_threshold = 100.0  # Ajustez selon vos besoins
segment_duration = 5.0  # Durée de chaque segment à analyser

# Supprimer les fragments à haute variance dans l'audio entier
aud, clean_audio = process_audio(audio, sr, threshold=variance_threshold, segment_duration=segment_duration)

# Sauvegarder l'audio filtré
sf.write("cleaned_audio_high_variance_removed.wav", clean_audio, sr)



##### comparaison des audios

In [ ]:
print("Écouter le signal original :")
display(Audio(aud, rate=sr))

print("Écouter le signal lissé (moyenne glissante) :")
display(Audio(clean_audio, rate=sr))

### approch 3: lisser la trame audio avec les hautes variances

In [ ]:


# Fonction pour lisser les fragments à haute variance
def smooth_high_variance_segments(audio, sr, frame_length=2048, hop_length=512, threshold=100.0):
    # Estimer le pitch avec pYIN
    pitches, voiced_flags, voiced_probs = librosa.pyin(audio, 
                                                       fmin=librosa.note_to_hz('C2'), 
                                                       fmax=librosa.note_to_hz('C7'),
                                                       sr=sr, hop_length=hop_length)
    
    # Récupérer les frames valides (où la voix est présente)
    valid_pitches = pitches[~np.isnan(pitches)]
    
    if len(valid_pitches) == 0:
        return audio
    
    pitch_variance = np.var(valid_pitches)
    
    if pitch_variance > threshold:
        # Si variance élevée, lisser la portion audio par interpolation linéaire
        t = np.arange(len(audio))
        valid_indices = np.where(~np.isnan(pitches))[0]
        valid_audio = audio[valid_indices]
        
        # Appliquer une interpolation linéaire sur les fragments invalides
        interpolator = interp1d(valid_indices, valid_audio, bounds_error=False, fill_value="extrapolate")
        smoothed_audio = interpolator(t)
        
        return smoothed_audio
    
    return audio

# Fonction pour traiter tout l'audio en segments et lisser les fragments à haute variance
def process_audio_smoothing(audio, sr, threshold=100.0, segment_duration=5.0):
    segment_samples = int(segment_duration * sr)
    num_segments = len(audio) // segment_samples
    
    processed_audio = np.array([])

    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = (i + 1) * segment_samples
        segment = audio[start_sample:end_sample]
        
        processed_segment = smooth_high_variance_segments(segment, sr, threshold=threshold)
        processed_audio = np.concatenate((processed_audio, processed_segment))
    
    return processed_audio

# Charger un fichier audio
audio_file = "your_audio_file.wav"
audio, sr = librosa.load(audio_file, sr=None)

# Appliquer le lissage sur les fragments à haute variance
variance_threshold = 100.0
segment_duration = 5.0

smoothed_audio = process_audio_smoothing(audio, sr, threshold=variance_threshold, segment_duration=segment_duration)

# Sauvegarder l'audio lissé
sf.write("smoothed_audio.wav", smoothed_audio, sr)
